<a href="https://colab.research.google.com/github/anishka-pateriya/mgmt467-analytics-portfolio/blob/main/Lab_VertexAI_BigQuery_PromptsOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Copy Schema to a dataframe

In [ ]:
# List datasets in the project
from google.cloud import bigquery

project_id = 'mgmt-467-25259'  # Explicitly define the project ID
client = bigquery.Client(project=project_id) # Initialize the client with the project ID
datasets = list(client.list_datasets())

if datasets:
    print("Datasets in project {}:".format(client.project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("No datasets found in project {}.".format(client.project))

In [ ]:
from google.cloud import bigquery
import pandas as pd
from google.api_core.exceptions import NotFound

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-25259'
dataset_id = 'superstore_data' # Use the confirmed dataset ID
table_id = 'sales' # Use the table name suggested in the markdown

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

try:
    # Check if the dataset exists
    dataset_ref = client.dataset(dataset_id, project=project_id)
    client.get_dataset(dataset_ref)  # This will raise NotFound if the dataset doesn't exist
    print(f"Dataset '{dataset_id}' found.")

    # Check if the table exists
    table_ref = dataset_ref.table(table_id)
    client.get_table(table_ref) # This will raise NotFound if the table doesn't exist
    print(f"Table '{table_id}' found in dataset '{dataset_id}'.")

    # Extract schema information if table exists
    schema_list = []
    table = client.get_table(table_ref)
    for field in table.schema:
        schema_list.append({
            'name': field.name,
            'field_type': field.field_type,
            'mode': field.mode,
            'description': field.description
        })

    # Convert to Pandas DataFrame
    schema_df = pd.DataFrame(schema_list)
    print("\nSchema DataFrame created:")
    # To see the output, run the code.

except NotFound:
    print(f"Error: Dataset '{dataset_id}' or Table '{table_id}' not found in project '{project_id}'.")
    print("\nAccording to the notebook instructions, you need to load the Global Superstore (Kaggle) dataset into BigQuery first.")
    print(f"Please create a dataset named '{dataset_id}' in your project '{project_id}' and load the data into a table named '{table_id}'.")
    print("Refer to the 'Dataset assumptions' section in the markdown cells above for more details.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # Exclude the problematic column
    if original_name == '记录数':
        continue

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a SQL query to select the first 10 rows from the view
    query_string = f"""
    SELECT
        *
    FROM
        `{project_id}.{dataset_id}.{new_view_id}`
    LIMIT 10;
    """

    # Execute the query
    query_job = client.query(query_string)

    # Wait for the query to complete and fetch results into a DataFrame
    rows_df = query_job.to_dataframe()

    # Print the DataFrame
    print(rows_df.to_markdown(index=False))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")

In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-25259.superstore_data.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

The result I came up with did match my expectations, although I did have to repeat through a few iterations of what I was doing. The initial prompt that was generated did not output the correct result, so I had to specify what format I wanted the output in.

In [ ]:
query_string = """
SELECT DISTINCT
    t1.sub_category AS Sub_Category -- Use the cleaned column name
FROM
    `mgmt-467-25259.superstore_data.superstore_clean` AS t1 -- Use the correct table path
WHERE
    t1.region = 'West' -- Use the cleaned column name
ORDER BY
    t1.sub_category -- Use the cleaned column name
LIMIT 100
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

The prompt outputted the code that I needed on the first try once making it runnable.

In [ ]:
query_string = """
SELECT
    t1.customer_id,
    SUM(t1.profit) AS total_profit
FROM
    `mgmt-467-25259.superstore_data.superstore_clean` AS t1 -- Use the correct table path
GROUP BY
    t1.customer_id
ORDER BY
    total_profit DESC
LIMIT 10
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
    t1.ship_mode, -- Use the cleaned column name
    COUNT(*) AS order_count
FROM
    `mgmt-467-25259.superstore_data.superstore_clean` AS t1 -- Use the correct table path
WHERE
    t1.category = 'Technology' -- Use the cleaned column name
GROUP BY
    t1.ship_mode
ORDER BY
    order_count DESC
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """
-- Compute monthly revenue for the last 12 full months.
-- Sort by year_month ascending.

SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(SAFE_CAST(Order_Date AS DATE), MONTH)) AS year_month,
    SUM(Sales) AS monthly_revenue
  FROM
    `mgmt-467-25259.superstore_data.superstore_clean` -- Use the correct table path
  WHERE
    SAFE_CAST(Order_Date AS DATE) >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 12 MONTH)
    AND SAFE_CAST(Order_Date AS DATE) < DATE_TRUNC(CURRENT_DATE(), MONTH)
  GROUP BY
    year_month
  ORDER BY
    year_month ASC
  LIMIT 1000
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

In [34]:
# Query to find the minimum and maximum order_date in the superstore_clean table
query_date_range = """
SELECT
    MIN(SAFE_CAST(SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%F', order_date) AS DATE)) AS min_order_date,
    MAX(SAFE_CAST(SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%F', order_date) AS DATE)) AS max_order_date
FROM
    `mgmt-467-25259.superstore_data.superstore_clean`;
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job_dates = client.query(query_date_range)

# Wait for the query to complete and fetch results into a DataFrame
results_df_dates = query_job_dates.to_dataframe()

# Display the DataFrame
print("Date range in the superstore_clean table:")
display(results_df_dates)

# Calculate and print the date range being filtered by the monthly revenue query
from datetime import date
from dateutil.relativedelta import relativedelta

today = date.today()
# Start of the current month
start_of_current_month = today.replace(day=1)
# Start of the 12 full months prior to the current month
start_of_filter_window = start_of_current_month - relativedelta(months=12)
# End of the filter window (exclusive - up to the start of the current month)
end_of_filter_window = start_of_current_month

print(f"\nDate range being filtered by the query (last 12 full months):")
print(f"Start Date (inclusive): {start_of_filter_window.strftime('%Y-%m-%d')}")
print(f"End Date (exclusive): {end_of_filter_window.strftime('%Y-%m-%d')}")

Date range in the superstore_clean table:


,min_order_date,max_order_date
0,NaT,NaT



Date range being filtered by the query (last 12 full months):
Start Date (inclusive): 2024-09-01
End Date (exclusive): 2025-09-01


In [32]:
query_string_sample_dates = """
SELECT
    order_date
FROM
    `mgmt-467-25259.superstore_data.superstore_clean`
LIMIT 10;
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job_sample_dates = client.query(query_string_sample_dates)

# Wait for the query to complete and fetch results into a DataFrame
sample_dates_df = query_job_sample_dates.to_dataframe()

# Display the DataFrame
print("Sample order_date values:")
display(sample_dates_df)

Sample order_date values:


,order_date
0,2011-10-03 00:00:00.000
1,2011-10-03 00:00:00.000
2,2011-12-26 00:00:00.000
3,2011-12-26 00:00:00.000
4,2011-12-26 00:00:00.000
5,2011-09-21 00:00:00.000
6,2011-09-21 00:00:00.000
7,2011-08-01 00:00:00.000
8,2011-08-01 00:00:00.000
9,2011-09-21 00:00:00.000


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

HAVING filters results after they have been grouped and aggregated, whereas WHERE filters individual rows before any grouping occurs.

In [37]:
query_string = """
SELECT
    sub_category, -- Use the cleaned column name
    SUM(profit) AS total_profit -- Use the cleaned column name
FROM
    `mgmt-467-25259.superstore_data.superstore_clean` -- Use the correct table path
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC
LIMIT 1000
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

,sub_category,total_profit
0,Tables,-64083.3887


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [40]:
query_string = """
SELECT
    product_id, -- Use cleaned column name
    product_name, -- Use cleaned column name
    SUM(sales) AS total_sales -- Use cleaned column name
FROM
    `mgmt-467-25259.superstore_data.superstore_clean` -- Use the correct table path
GROUP BY
    product_id,
    product_name
ORDER BY
    total_sales DESC
LIMIT 1000
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61600.0
1,TEC-PH-10004664,"Nokia Smart Phone, with Caller ID",30042.0
2,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27454.0
3,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.0
4,TEC-PH-10004823,"Nokia Smart Phone, Full Size",22261.0
...,...,...,...
995,OFF-BI-10004318,Ibico EB-19 Dual Function Manual Binding System,3236.0
996,FUR-CH-10004331,"SAFCO Rocking Chair, Red",3235.0
997,TEC-MA-10000875,"Okidata Inkjet, Wireless",3235.0
998,OFF-ST-10002561,"Rogers File Cart, Blue",3226.0


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

In [42]:
query_string = """
WITH
  -- CTE 1: Calculate total sales for each State within each Region
  state_sales AS (
    SELECT
      t1.region, -- Use cleaned column name
      t1.state, -- Use cleaned column name
      SUM(t1.sales) AS total_sales -- Use cleaned column name
    FROM
      `mgmt-467-25259.superstore_data.superstore_clean` AS t1 -- Use the correct table path
    GROUP BY
      t1.region,
      t1.state
  ),
  -- CTE 2: Rank states by total sales within each Region
  ranked_state_sales AS (
    SELECT
      *,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- Final SELECT: Filter for the top 3 ranked states per region
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  region,
  sales_rank
LIMIT 1000
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

,region,state,total_sales,sales_rank
0,Africa,Gauteng,51608.0,1
1,Africa,Kinshasa,42533.0,2
2,Africa,Al Qahirah,38436.0,3
3,Canada,Ontario,35451.0,1
4,Canada,Quebec,10928.0,2
5,Canada,British Columbia,9546.0,3
6,Caribbean,Santo Domingo,78710.0,1
7,Caribbean,Santiago de Cuba,32456.0,2
8,Caribbean,Granma,16997.0,3
9,Central,Ile-de-France,317818.0,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

The primary failure modes are missing data for one of the two years and inconsistent sub-category names. The first issue, where a sub-category has sales in only 2023 or 2024, causes the INNER JOIN to exclude it, which can be fixed by switching to a FULL OUTER JOIN and using COALESCE to substitute NULL sales with 0. The second issue, involving misspellings like "Table" vs. "Tables," prevents records from joining correctly; this requires a data standardization step—such as a CASE statement—to harmonize the Sub_Category names before the join is performed.

In [44]:
query_string = """
WITH
  -- CTE to calculate total sales by Sub-Category and Year
  yr_sales AS (
    SELECT
      sub_category, -- Use cleaned column name
      EXTRACT(YEAR FROM SAFE_CAST(SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%F', order_date) AS TIMESTAMP)) AS sales_year, -- Cast and extract year
      SUM(sales) AS total_sales -- Use cleaned column name
    FROM
      `mgmt-467-25259.superstore_data.superstore_clean` -- Use the correct table path
    WHERE
      SAFE_CAST(SAFE.PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%F', order_date) AS TIMESTAMP) IS NOT NULL -- Filter out unparseable dates
    GROUP BY
      1,
      2
  )
-- Final SELECT: Pivot the data to calculate the YoY delta
SELECT
  t2024.sub_category,
  t2023.total_sales AS sales_2023,
  t2024.total_sales AS sales_2024,
  (t2024.total_sales - t2023.total_sales) AS yoy_delta
FROM
  yr_sales AS t2024
  INNER JOIN yr_sales AS t2023
    ON t2024.sub_category = t2023.sub_category
WHERE
  t2024.sales_year = 2024
  AND t2023.sales_year = 2023
ORDER BY
  yoy_delta DESC
LIMIT 5
"""

# Assuming 'client' is already initialized from previous cells
# Execute the query
query_job = client.query(query_string)

# Wait for the query to complete and fetch results into a DataFrame
results_df = query_job.to_dataframe()

# Display the DataFrame
display(results_df)

,sub_category,sales_2023,sales_2024,yoy_delta


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---